In [ ]:
import csv
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from numpy import argmax

# 데이터 로딩 및 전처리
X = []
Y = []
try:
    with open('[Dataset]_Module25_disasters_social_media.csv', 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['text'] and row['choose_one']:
                X.append(row['text'].strip())
                Y.append(row['choose_one'].strip())
except FileNotFoundError:
    print("파일을 찾을 수 없어 예시 데이터를 사용합니다.")
    X = ['Hi', 'Hello', 'How are you?', 'I am making', 'making', 'working', 'studying', 'see you later', 'bye', 'goodbye']
    Y = ['greeting', 'greeting', 'greeting', 'busy', 'busy', 'busy', 'busy', 'bye', 'bye', 'bye']

def preprocess_data(data):
    data = [d.lower() for d in data]
    data = [re.sub('[^a-z\s]', '', d) for d in data]
    data = [d.strip() for d in data]
    data = [re.sub(' +', ' ', d) for d in data]
    return data

X = preprocess_data(X)
vocabulary = sorted(list(set(' '.join(X).split())))
classes = sorted(list(set(Y)))

def encode_sentence(sentence, vocab):
    sentence = preprocess_data([sentence])[0]
    encoded_vector = [0] * len(vocab)
    for word in sentence.split(' '):
        try:
            encoded_vector[vocab.index(word)] = 1
        except ValueError:
            pass
    return encoded_vector

X_encoded = [encode_sentence(sentence, vocabulary) for sentence in X]
Y_encoded = []
for label in Y:
    label_encoded = [0] * len(classes)
    label_encoded[classes.index(label)] = 1
    Y_encoded.append(label_encoded)

# 모델 생성 및 훈련
model = Sequential()
model.add(Dense(units=64, activation='sigmoid', input_dim=len(X_encoded[0])))
model.add(Dense(units=len(Y_encoded[0]), activation='softmax'))
model.compile(loss=categorical_crossentropy, optimizer=SGD(learning_rate=0.01, momentum=0.9, nesterov=True), metrics=['accuracy'])

print("모델 학습을 시작합니다. 잠시만 기다려 주세요...")

history = model.fit(np.array(X_encoded), np.array(Y_encoded), 
                    epochs=100, 
                    batch_size=16, 
                    verbose=1,
                    validation_split=0.2)

# 학습 곡선 시각화
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Curve')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 모델 평가
predictions = model.predict(np.array(X_encoded), verbose=0)
predicted_classes = [classes[argmax(p)] for p in predictions]
true_classes = [classes[argmax(p)] for p in Y_encoded]

print("\n" + "=" * 50)
print("모델의 최종 평가 지표:")
print(classification_report(true_classes, predicted_classes, zero_division=0))

# 모델 및 관련 데이터 저장
model.save('model.h5')
with open('data.pkl', 'wb') as f:
    pickle.dump({'vocabulary': vocabulary, 'classes': classes}, f)

print("\n모델 학습 및 저장이 완료되었습니다. 'model.h5'와 'data.pkl' 파일이 생성되었습니다.")

In [ ]:
import numpy as np
import re
from keras.models import load_model
from numpy import argmax
import pickle
import random

# 모델 및 관련 데이터 불러오기
try:
    model = load_model('model.h5')
    with open('data.pkl', 'rb') as f:
        data = pickle.load(f)
    vocabulary = data['vocabulary']
    classes = data['classes']
except (OSError, FileNotFoundError, KeyError):
    print("모델 파일이 없거나 손상되었습니다. 먼저 모델 학습 코드를 실행해 주세요.")
    exit()

# 텍스트 전처리 및 인코딩 함수 (모델 학습 코드와 동일)
def preprocess_data(data):
    data = [d.lower() for d in data]
    data = [re.sub('[^a-z\s]', '', d) for d in data]
    data = [d.strip() for d in data]
    data = [re.sub(' +', ' ', d) for d in data]
    return data

def encode_sentence(sentence, vocab):
    sentence = preprocess_data([sentence])[0]
    encoded_vector = [0] * len(vocab)
    for word in sentence.split(' '):
        try:
            encoded_vector[vocab.index(word)] = 1
        except ValueError:
            pass
    return encoded_vector

# 챗봇 테스트 루프
print("챗봇이 준비되었습니다. '종료'를 입력하면 멈춥니다.")
while True:
    sentence = input("문장을 입력하세요: ")
    if sentence.lower() == '종료':
        print("챗봇을 종료합니다.")
        break
    
    encoded_input = np.array([encode_sentence(sentence, vocabulary)])
    prediction = model.predict(encoded_input, verbose=0)
    
    predicted_class_index = argmax(prediction)
    predicted_class = classes[predicted_class_index]
    
    # 예측된 의도에 따라 답변 출력
    if predicted_class == 'Relevant':
        responses = ["이것은 재난 관련 트윗으로 분류됩니다.", "재난 관련 정보인 것 같습니다.", "긴급한 상황인 것 같군요."]
    elif predicted_class == 'Not Relevant':
        responses = ["이것은 재난과 관련 없는 트윗으로 분류됩니다.", "재난 정보가 아닌 것 같습니다.", "일반적인 정보로군요."]
    else:
        responses = ["이해하지 못했어요.", "무슨 말씀이신지 잘 모르겠어요."]
        
    print(random.choice(sentence, responses))